# **Aperture photometry**


_**Developed by**_  <br>
Raghav Wani <br>
Vivek Jha

_**Under guidance of**_ <br>
Dr. Yogesh Joshi <br>
Aryabhatta Research Institute of Observational Sciences (ARIES), Nainital

In [ ]:
import numpy as np
import matplotlib as mp
import pandas as pd
import math, re, random
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import AutoMinorLocator
import ccdproc,os,sys,time,random, csv, os
import warnings
import astroalign as aa
from glob import glob
from itertools import combinations
#ASTROPY
from astropy import units as u
from astropy.io import fits
from astropy.time import Time
from astropy.wcs import WCS
from astropy.nddata import CCDData
from astropy.coordinates import SkyCoord, Angle
from astropy import coordinates as coord
from astropy.stats import sigma_clipped_stats, SigmaClip
from astropy.visualization import ImageNormalize, LogStretch, ZScaleInterval, SqrtStretch, simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.stats import SigmaClip, mad_std
from astroquery.simbad import Simbad
#PHOTUTILS
from photutils.datasets import make_100gaussians_image
from photutils.background import Background2D, MeanBackground,SExtractorBackground
from photutils import find_peaks, CircularAperture, CircularAnnulus, aperture_photometry
from photutils import Background2D, MedianBackground, DAOStarFinder
from photutils.utils import calc_total_error

#### **Folder name**
folder where all the images are stored

In [ ]:
tmp_dir = '/home/aries/projetFolder/NGC6709/NGC6709_1/20211009/' #should start and end with "/"
warnings.filterwarnings("ignore") #ignore warnings

In [ ]:
#To check details of the frame
files=sorted(glob.glob(os.path.join(tmp_dir, 'S-*cleaned.fits'))) #list of science frame path. 
print(len(files))
for i in range(len(files)):
    data_0,header_0=fits.getdata(files[i],header=True)
    print(header_0['RA'], header_0['DEC'])

####  **Clearing unwanted data from the night of observation**

In [ ]:
files = os.listdir(tmp_dir)

start = time.time()
for file_name in files:
    file_path = os.path.join(tmp_dir, file_name)

    #change starts with as per target file:
    if file_name.endswith('.fits') and file_name.startswith('F-2022'): 
        
        with fits.open(file_path) as hdul:
            header = hdul[0].header
        # keyword used for sorting:
        object_value = header.get('FILTER1') #change as per target file
        #for S- use: ORIGFILE
        #for F- use: FILTER1

        # Delete the file 
        if not object_value.startswith('R'): #might change as per header
            os.remove(file_path)
            print(f"Deleted file: {file_name}")
            
end = time.time()
print(end - start)

### **Cleaning**

###### A. Creating Master bias frame 

In [ ]:
start = time.time()
warnings.filterwarnings("ignore")

bias_files = sorted(glob(os.path.join(tmp_dir, "BIAS*.fits")))
biaslist = []
print(bias_files)
for i in range (len(bias_files)):
    data= ccdproc.CCDData.read(bias_files[i],unit='adu')
    print(data.data.shape)
    biaslist.append(data)
    
masterbias = ccdproc.combine(biaslist,method='median',sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                             sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std)

masterbias.write(tmp_dir+ 'masterbias.fits', overwrite=True)

mbias=ccdproc.CCDData.read(tmp_dir + 'masterbias.fits', unit='adu')
print('Master bias generated')
print(" Mean and median of the masterbias:", np.mean(masterbias), np.median(masterbias))

end = time.time()
print('Execution took:', end - start, 'seconds')

###### B. Creating Master flat frame

In [ ]:
start = time.time()
warnings.filterwarnings("ignore")

flat_files=sorted(glob(os.path.join(tmp_dir, "F-*.fits")))
flatlist=[]
for j in range(len(flat_files)):
    flat=ccdproc.CCDData.read(flat_files[j],unit='adu')
    mbias=ccdproc.CCDData.read(tmp_dir + 'masterbias.fits',unit='adu')
    print(mbias.data.shape)
    print(flat.data.shape)
    flat_bias_removed=ccdproc.subtract_bias(flat,mbias) #flat - master bias
    flatlist.append(flat_bias_removed)

# normalizing masterflat:
def inv_median(a):
    return 1 / np.median(a)

masterflat = ccdproc.combine(flatlist,method='median',scale=inv_median ,sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5, sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std)
masterflat.write(tmp_dir+ 'masterflat.fits', overwrite=True)
mflat=ccdproc.CCDData.read(tmp_dir + 'masterflat.fits',unit='adu')
print('Master flat generated')
print("Mean and median of the masterflat: ", np.mean(masterflat), np.median(masterflat))

end = time.time()
print('Execution took:', end - start, 'seconds')

###### C. Master bias substraction, flat-fielding, cosmis rays removal

In [ ]:
#FINAL CODE FOR CLEANING:
start = time.time()
warnings.filterwarnings("ignore")

files=sorted(glob(os.path.join(tmp_dir, "S-*.fits")))
for i in range(len(files)): #(0,len(files))
    print(files[i])
    image=ccdproc.CCDData.read(files[i],unit='adu')
    header=fits.getheader(files[i],0)
    
    mbias=ccdproc.CCDData.read(tmp_dir+ 'masterbias.fits',unit='adu') 
    mflat=ccdproc.CCDData.read(tmp_dir+ 'masterflat.fits',unit='adu')
    #shape should be 2D, if not use '[0]'
    print(mbias.data.shape) #(2048, 2048)
    print(mflat.data.shape) #(2048, 2048)
    print(image.data.shape) #(2048, 2048)
    
    if image.data.shape == mbias.data.shape: #to avoid kinetic mode data
        bias_subtracted = ccdproc.subtract_bias(image, mbias)
        flat_corrected = ccdproc.flat_correct(bias_subtracted, mflat)
        
        cr_cleaned = ccdproc.cosmicray_lacosmic(flat_corrected,readnoise=10, sigclip=5, verbose=True)
        print('Cosmic rays removed')
        clean_file=files[i].replace('.fits','')
    
        fits.writeto(clean_file+'_cleaned.fits',cr_cleaned, header=header,overwrite=True)
        print('Image no',i+1,'has been cleaned')
        print(" Mean and median of the cleaned image: ",np.mean(cr_cleaned), np.median(cr_cleaned))  

end = time.time()
print('Execution took:', end - start, 'seconds')

###### D. Plotting cleaned images

In [ ]:
#Plot of science frame

warnings.filterwarnings("ignore")

tmp_dir = tmp_dir = '/home/aries/projetFolder/NGC6709/NGC6709_2/20211105/'
files = sorted(glob.glob(os.path.join(tmp_dir, "S-*.fits")))
print(files[0])


hdul = fits.open(files[0])
image_data = hdul[0].data
wcs = WCS(hdul[0].header)

# Display the image with reverse gray scale
interval = ZScaleInterval()
vmin, vmax = interval.get_limits(image_data)

fig = plt.figure()
ax = fig.add_subplot(111, projection=wcs)

im = ax.imshow(image_data, cmap='gray_r', origin='lower', vmin=vmin, vmax=vmax)
cbar = fig.colorbar(im, ax=ax)

ax.set_xlabel('Right ascension')
ax.set_ylabel('Declination')

ax.tick_params(axis='both', direction='in', width=1.5, length=5, labelsize=8)#, top=True, left=True, right=True)

x_ticks_pix = ax.get_xticks()
y_ticks_pix = ax.get_yticks()

x_ticks_world, y_ticks_world = wcs.wcs_pix2world(x_ticks_pix, y_ticks_pix, 0)

ra_ticks = Angle(x_ticks_world, unit='deg').to_string(unit=u.hourangle, sep=':')
ax.set_xticklabels(ra_ticks, fontweight='bold')

dec_ticks = Angle(y_ticks_world, unit='deg').to_string(unit=u.degree, sep=':')
ax.set_yticklabels(dec_ticks, fontweight='bold')
plt.title("NGC6709 Uncleaned image")
plt.show()
hdul.close()

In [ ]:
#Plot of Flat and Bias frames

warnings.filterwarnings("ignore")

tmp_dir = '/home/aries/projetFolder/NGC6709/NGC6709_2/20211105/'

files = sorted(glob(os.path.join(tmp_dir, "F-*.fits")))
print(files[0])
hdul = fits.open(files[0])
image_data = hdul[0].data
wcs = WCS(hdul[0].header)

interval = ZScaleInterval()
vmin, vmax = interval.get_limits(image_data)

fig = plt.figure()
ax = fig.add_subplot(111, projection=wcs)

im = ax.imshow(image_data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
cbar = fig.colorbar(im, ax=ax)

ax.set_xlabel('X axis pixel')
ax.set_ylabel('Y axis pixel')

ax.tick_params(axis='both', direction='in', width=1.5, length=5, labelsize=8)#, top=True, left=True, right=True)

x_ticks_pix = ax.get_xticks()
y_ticks_pix = ax.get_yticks()

x_ticks_world, y_ticks_world = wcs.wcs_pix2world(x_ticks_pix, y_ticks_pix, 0)

ra_ticks = Angle(x_ticks_world, unit='deg').to_string(unit=u.hourangle, sep=':')
ax.set_xticklabels(ra_ticks, fontweight='bold')

dec_ticks = Angle(y_ticks_world, unit='deg').to_string(unit=u.degree, sep=':')
ax.set_yticklabels(dec_ticks, fontweight='bold')
plt.title("Flat frame")
plt.show()
hdul.close()

### **Aperture photometry**

###### A. Definitions:

In [ ]:
#For source detection:
#'files' is the list of path of all the cleaned frames.
# 'n' is the index number of particular cleaned frame in 'files'
def source(files,n):
    data,header=fits.getdata(files[n] ,header=True)
    sigma_clip = SigmaClip(sigma=3, maxiters=10)
    bkg_estimator = SExtractorBackground()
    bkg = Background2D(data, (10,10), filter_size=(3, 3),sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
    back=bkg.background
    h = fits.open(filename)
    wcs = WCS(h[0].header)
    mask = data == 0
    unit = u.electron / u.s

    xdf_image = CCDData(data, unit=unit, meta=header, mask=mask)
    norm_image = ImageNormalize(vmin=1e-4, vmax=5e-2, stretch=LogStretch(), clip=False)
    xdf_image_clipped = np.clip(xdf_image, 1e-4, None)

    mean, median, std = sigma_clipped_stats(xdf_image.data, sigma=3.0, maxiters=20, mask=xdf_image.mask)
    daofind = DAOStarFinder(fwhm=4, threshold=5*std)
    return daofind(data - back)

In [ ]:
# For getting magnitude and magnitude error:
#apertures is the variable that creates aperture of given radius and shape.
#an_ap is the variable that creates annulus of given inner and outer radii and its shape.
def magnitude(files, n,apertures, an_ap ):
    data,header=fits.getdata(files[n] ,header=True)
    sigma_clip = SigmaClip(sigma=3, maxiters=10)
    bkg_estimator = SExtractorBackground()    
    bkg = Background2D(data, (10,10), filter_size=(3, 3),sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
    back=bkg.background
    exposure=header['EXPTIME']
    effective_gain=exposure
    error=calc_total_error(data,back,effective_gain)

    phot_table = aperture_photometry(data-back, apertures,error=error)
    phot_table2=aperture_photometry(data-back,an_ap)

    bkg_mean = phot_table2['aperture_sum'] / an_ap.area
    bkg_sum = bkg_mean * an_ap.area

    final_sum0=phot_table['aperture_sum_0']-bkg_sum
    final_sum1=phot_table['aperture_sum_1']-bkg_sum
    final_sum2=phot_table['aperture_sum_2']-bkg_sum
    final_sum3=phot_table['aperture_sum_3']-bkg_sum
    final_sum4=phot_table['aperture_sum_4']-bkg_sum
            
    mag_back=-2.5*np.log10(bkg_mean/exposure)
    mag_0=-2.5*np.log10(final_sum0/exposure)
    mag_1=-2.5*np.log10(final_sum1/exposure)
    mag_2=-2.5*np.log10(final_sum2/exposure)
    mag_3=-2.5*np.log10(final_sum3/exposure)
    mag_4=-2.5*np.log10(final_sum4/exposure)

    flux_err_0=phot_table['aperture_sum_err_0']
    mag_err_0=1.09*flux_err_0/final_sum0

    flux_err_1=phot_table['aperture_sum_err_1']
    mag_err_1=1.09*flux_err_1/final_sum1

    flux_err_2=phot_table['aperture_sum_err_2']
    mag_err_2=1.09*flux_err_2/final_sum2

    flux_err_3=phot_table['aperture_sum_err_3']
    mag_err_3=1.09*flux_err_3/final_sum3

    flux_err_4=phot_table['aperture_sum_err_4']
    mag_err_4=1.09*flux_err_4/final_sum4 
    return mag_0,mag_err_0,mag_1,mag_err_1,mag_2,mag_err_2 ,mag_3,mag_err_3,mag_4,mag_err_4

###### **B. Plotting aperture**

In [ ]:
files=sorted(glob(os.path.join('/home/aries/projetFolder/NGC6709/NGC6709_1/20211009/S*cleaned.fits')))

f=0
filename= files[f]
sources = source(files, f)

positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=7)
annulus_aperture = CircularAnnulus(positions, r_in=14, r_out=15)
norm = simple_norm(data, 'linear', 'cool', percent=99)


plt.xlim(300, 600)
plt.ylim(1000, 1200)

plt.imshow(data, norm=norm, interpolation='nearest')
apertures.plot(color='blue', lw=1, alpha=0.5)
ap_patches = apertures.plot(color='white', lw=0.5)
ann_patches = annulus_aperture.plot(color='green', lw=0.05)
handles = (ap_patches[0], ann_patches[0])
plt.xlabel('X coordinate of frame')
plt.ylabel('Y coordinate of frame')
plt.show()

###### B. Optimum aperture:
Growth curve

In [ ]:
# CREATING MAGNITUDE APERTURE FILE FOR single FRAME

start = time.time()
warnings.filterwarnings("ignore")

files=sorted(glob(os.path.join(tmp_dir, "S-*cleaned.fits")))

f = 0
source_0 = source(files , f)
print('No. of sources: ',len(source_0))
print(source_0)

radii = [5, 6, 7, 8, 9]
positions = [(source_0['xcentroid'][i], source_0['ycentroid'][i]) for i in range(len(source_0))]
apertures = [CircularAperture(positions, r=r) for r in radii]
an_ap = CircularAnnulus(positions, r_in=14, r_out=15)

mag_0, mag_err_0, mag_1, mag_err_1, mag_2, mag_err_2, mag_3, mag_err_3, mag_4, mag_err_4 = magnitude(files, f,apertures, an_ap)
jd = header_0['JD']
jd_list = [jd] * len(source_0['xcentroid'])
h = fits.open(files[f])
wcs = WCS(h[0].header)
ra , dec = wcs.all_pix2world(source_0['xcentroid'],source_0['ycentroid'],0)
print(ra, dec)

with open(f"{tmp_dir}zipped_data_{f}.csv", 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['JD', 'RA', 'DEC', 'Magnitude0', 'Magnitude_error0', 'Magnitude1', 'Magnitude_error1', 'Magnitude2', 'Magnitude_error2', 'Magnitude3', 'Magnitude_error3', 'Magnitude4', 'Magnitude_error4'])
    writer.writerows(zip(jd_list, ra,dec, mag_0, mag_err_0, mag_1, mag_err_1, mag_2, mag_err_2, mag_3, mag_err_3, mag_4, mag_err_4))

end = time.time()
print('Execution took:', end - start, 'seconds')

In [ ]:
# PLOTING "growth curve" TO CHOSE OPTIMUM APERTURE SIZE
x=radii
start = time.time()
plt.figure(figsize = (8,8))
for i in range(20,40):
    y=[]
    y.append(mag_4[i]-mag_0[i])
    y.append(mag_4[i]-mag_1[i])
    y.append(mag_4[i]-mag_2[i])
    y.append(mag_4[i]-mag_3[i])
    y.append(mag_4[i]-mag_4[i])  
    plt.plot(x,y, 'ro-',markersize=2, lw=0.2, label='%i'%i)
    #plt.ylim(-0.5,1)
    plt.xlabel('Aperture diameter (pixels)', fontsize=15)
    plt.ylabel('Largest aperture mag - aperture mag', fontsize = 15)
    plt.xticks(fontsize = 15)
    plt.yticks(fontsize = 15)
#plt.legend()
plt.title('Growth curve for 5 radii')
plt.show()
    
end = time.time()
print('Execution took: ',end - start,' seconds')

In [ ]:
# CREATING MAGNITUDE APERTURE FILE FOR ALL FRAMES

start = time.time()
warnings.filterwarnings("ignore")

files=sorted(glob(os.path.join(tmp_dir, "S-*cleaned.fits")))

radii = [5,6,7,8,9]
for f in range(len(files)):
    filename=files[f]
    
    print(files[f])
    source_0 = source(files , f)
    print('No. of sources:',len(source_0))

    positions = [(source_0['xcentroid'][i], source_0['ycentroid'][i]) for i in range(len(source_0))]
    apertures = [CircularAperture(positions, r=r) for r in radii]
    an_ap = CircularAnnulus(positions, r_in=14, r_out=15)

    mag_0, mag_err_0, mag_1, mag_err_1, mag_2, mag_err_2, mag_3, mag_err_3, mag_4, mag_err_4 = magnitude(files,f, apertures, an_ap)
    jd = header_0['JD']
    jd_list = [jd] * len(source_0['xcentroid'])
    h = fits.open(files[f])
    wcs = WCS(h[0].header)
    ra , dec = wcs.all_pix2world(source_0['xcentroid'],source_0['ycentroid'],0)
    
    with open(f"{tmp_dir}zipped_data/zipped_data_{f}.csv", 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['JD' ,'RA', 'DEC', 'Magnitude0', 'Magnitude_error0', 'Magnitude1', 'Magnitude_error1', 'Magnitude2', 'Magnitude_error2', 'Magnitude3', 'Magnitude_error3', 'Magnitude4', 'Magnitude_error4'])
        writer.writerows(zip(jd_list, ra,dec, mag_0, mag_err_0, mag_1, mag_err_1, mag_2, mag_err_2, mag_3, mag_err_3, mag_4, mag_err_4))
    
    print('file',f+1,'generated')
end = time.time()
print('Execution took: ',end - start,' seconds')

###### C. Calibration of Magnitude from simbad:

In [ ]:
#SIMBAD CODE
start = time.time()
warnings.filterwarnings("ignore")

# Read RA and DEC from the text file
star_coordinates = []
with open(f"{tmp_dir}zipped_data/zipped_data_0.csv", 'r') as file: #
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
        ra, dec = row[1], row[2]
        star_coordinates.append((float(ra), float(dec)))

# Query Simbad for magnitudes of stars
Simbad.reset_votable_fields()
Simbad.add_votable_fields('flux(R)', 'flux_error(R)')

ra_list = []
dec_list = []
magnitude_list = []
magnitude_error_list = []
print('reached 1')
for ra, dec in star_coordinates:
    coords = SkyCoord(ra=ra, dec=dec, unit=u.deg, frame='icrs')
    result_table = Simbad.query_region(coords, radius=5 * u.arcsec)
    if result_table is not None:
        magnitude = result_table['FLUX_R'][0]
        magnitude_error = result_table['FLUX_ERROR_R'][0]
        ra_list.append(ra)
        dec_list.append(dec)
        magnitude_list.append(magnitude)
        magnitude_error_list.append(magnitude_error)
    else:
        ra_list.append(ra)
        dec_list.append(dec)
        magnitude_list.append('Notfound')
        magnitude_error_list.append('Notfound')
        
    print('done')
print('reached 2')

# Save the retrieved magnitudes to a .csv file
output_file = f"{tmp_dir}simbad_magnitudes.csv"
with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['RA', 'DEC', 'Magnitude', 'Magnitude_error'])
    writer.writerows(zip(ra_list, dec_list, magnitude_list, magnitude_error_list))

print(f"Star magnitudes saved to '{output_file}'.")
end = time.time()
print('Execution took: ',end - start,' seconds')

 NOTE: RUNNING SIMBAD CODE IS LUCK, SOMETIMES IT DOESN'T WORK AND MANYTIMES IT WORKS BUT BELOW CODE DON'T GIVE LINEAR PLOT! BE PATIENT...! 

$y = mx+c$ equation of best fit line. Histogram of intercept $c$

In [ ]:

start = time.time()
warnings.filterwarnings("ignore")

# Read the CSV files
file1 = pd.read_csv(f"{tmp_dir}zipped_data/zipped_data_0.csv") #reference frame file
file2 = pd.read_csv(f"{tmp_dir}simbad_magnitudes.csv") #simbad code generated file

# Extract the desired columns
y_values = file2['Magnitude']  
y_error = file2['Magnitude_error'] 

x_values2 = file1['Magnitude2']  
x_error2 = file1['Magnitude_error2']

# Convert y_values and y_error to numeric, handling invalid or missing values
y_values = pd.to_numeric(y_values, errors='coerce')
y_error = pd.to_numeric(y_error, errors='coerce')

# Filter out invalid or missing values from x_values, y_values, and y_error
valid_indices = (~np.isnan(x_values0)) & (~np.isnan(y_values)) & (~np.isnan(y_error))

x_values2 = x_values2[valid_indices]
x_error2 = x_error2[valid_indices]
y_values = y_values[valid_indices]
y_error = y_error[valid_indices]

print('Number of sources detected by simabd:', len(x_values2)) #NUMBER OF SOURCES DETECTED BY SIMBAD

# Plotting with errorbar
plt.errorbar(x_values2, y_values,xerr=x_error2, yerr=y_error, color='blue',fmt='.', label='Data')

slope2, intercept2 = np.polyfit(x_values2, y_values, 1)
best_fit_line2 = slope2 * x_values2 + intercept2
# Plot the best-fit line
plt.plot(x_values2, best_fit_line2, color='blue', label='Best Fit Line 2')
equation2 = f'y = {slope2:.2f}x + {intercept2:.2f}'

plt.text(0.5, 0.85, equation2,color = 'blue', ha='center', va='center', transform=plt.gca().transAxes)
plt.xlabel('Magnitude from code') 
plt.ylabel('Magnitude from simbad')  
plt.title('Calibration of magnitude') 
plt.legend()
plt.show()

mag=y_values - x_values2
plt.hist(mag, bins=25)
plt.xlabel('Calibration Magnitude')
plt.title('Histogram to correctly pick calibration magnitude')
plt.show()

In [ ]:
file1 = pd.read_csv(f'{tmp_dir}zipped_data/zipped_data_0.csv')  # reference frame file
file2 = pd.read_csv(f'{tmp_dir}simbad_magnitudes.csv')  # simbad code generated file

x_values2 = file1['Magnitude2']  
y_values = file2['Magnitude'] 

y_values = pd.to_numeric(y_values, errors='coerce')

valid_indices = (~np.isnan(x_values2)) & (~np.isnan(y_values))
x_values2 = x_values2[valid_indices]
y_values = y_values[valid_indices]

slope2, intercept2 = np.polyfit(x_values2, y_values, 1)
best_fit_line2 = slope2 * x_values2 + intercept2
fig, ax = plt.subplots(figsize=(8, 6))
counts, x_bins, y_bins, im = ax.hist2d(x_values2, y_values, bins=25,cmap='Reds')

ax.plot(x_values2, best_fit_line2, color='red', label='Best Fit Line')

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel('Density')

ax.set_xlabel('DFOT Magnitude Values')
ax.set_ylabel('SIMBAD Magnitude Values')
ax.set_title('Calibration of magnitude (Contour Plot)')


ax.legend()
plt.show()

###### D. Redefining and Calibrating 

In [ ]:
#redefined_magnitude()
#This is newly defined magnitude fuunction which 

def redefined_magnitude(files,n, apertures, an_ap ):
    data,header=fits.getdata(files[n] ,header=True)
    sigma_clip = SigmaClip(sigma=3, maxiters=10)
    bkg_estimator = SExtractorBackground()    
    bkg = Background2D(data, (10,10), filter_size=(3, 3),sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
    back=bkg.background
    exposure=header['EXPTIME']
    effective_gain=exposure
    error=calc_total_error(data,back,effective_gain)

    phot_table = aperture_photometry(data-back, apertures,error=error)
    phot_table2=aperture_photometry(data-back,an_ap)

    bkg_mean = phot_table2['aperture_sum'] / an_ap.area
    bkg_sum = bkg_mean * an_ap.area

    final_sum0=phot_table['aperture_sum_0']-bkg_sum         
    mag_back=-2.5*np.log10(bkg_mean/exposure)
    mag_0=-2.5*np.log10(final_sum0/exposure)
    flux_err_0=phot_table['aperture_sum_err_0']
    mag_err_0=1.09*flux_err_0/final_sum0
    return mag_0 , mag_err_0 

In [ ]:
#Calibrating reference frame (single)
#change def of magnitude before running this code!!

start = time.time()
warnings.filterwarnings("ignore")

files=sorted(glob(os.path.join(tmp_dir, "S-*cleaned.fits")))

f = 0
print(files[f])
source_0 = source(files,f)
print('No. of sources: ',len(source_0))
#-----------------------------------------------------------------
intercept= 22.11 #needs to be changed as per above plot
slope =1 #change only if error is more than 10-15%

radius = 7 #chose optimum aperture size
positions = [(source_0['xcentroid'][i], source_0['ycentroid'][i]) for i in range(len(source_0))]
apertures = CircularAperture(positions, r=radius)
an_ap = CircularAnnulus(positions, r_in=14, r_out=15)

mag_0, mag_err_0 = redefined_magnitude(files,f,apertures, an_ap)

calibrated_mag = slope*mag_0 + intercept #calibration step

jd = header_0['JD']
jd_list = [jd] * len(source_0['xcentroid'])
h = fits.open(files[f])
wcs = WCS(h[0].header)
ra , dec = wcs.all_pix2world(source_0['xcentroid'],source_0['ycentroid'],0)
print(ra, dec)
data = np.array([jd_list, ra, dec, mag_0, mag_err_0])
transposed_data = np.transpose(data)

with open(f"{tmp_dir}Calibrated_zipped_data_{f}.csv", 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['JD', 'RA', 'DEC', 'Calibrated_Magnitude', 'Magnitude_error0'])
    writer.writerows(zip(jd_list, ra,dec, calibrated_mag, mag_err_0))

end = time.time()
print('Execution took:', end - start, 'seconds')

In [ ]:
#PLOTING HISTOGRAM ALONG WITH BRIGHT SOURCE BOUNDARY
# Read the CSV file
data = pd.read_csv(f"{tmp_dir}Calibrated_zipped_data_{f}.csv")

column_3_data = data['Calibrated_Magnitude']  

# Plotting the histogram
plt.hist(column_3_data, bins=100)  # You can adjust the number of bins as per your preference
plt.axvline(x=17, color='r', linestyle='--', label='x=17')

# Add labels and title to the plot
plt.xlabel('Values')  
plt.ylabel('Frequency')  
plt.title('Histogram of Calibrated magnitude (ref) Data')  #
plt.legend()
plt.show()

In [ ]:
# CREATING MAGNITUDE APERTURE FILE FOR ALL FRAMES

start = time.time()
warnings.filterwarnings("ignore")
files = sorted(glob(os.path.join(tmp_dir, "S-*cleaned.fits")))

radius = 7      #chose optimum aperture size
intercept= 22.11 #needs to be changed as per above plot
slope = 1
for f in range(len(files)):

    print(files[f])
    source_0 = source(files , f)
    print('No. of sources: ',len(source_0))

    positions = [(source_0['xcentroid'][i], source_0['ycentroid'][i]) for i in range(len(source_0))]
    apertures = CircularAperture(positions, r=radius)
    an_ap = CircularAnnulus(positions, r_in=14, r_out=15)

    mag_0, mag_err_0 = redefined_magnitude(files, f,apertures, an_ap)
    
    calibrated_mag = slope*mag_0 + intercept
    
    jd = header_0['JD']
    jd_list = [jd]*len(source_0['xcentroid'])
    h = fits.open(files[f])
    wcs = WCS(h[0].header)
    ra , dec = wcs.all_pix2world(source_0['xcentroid'],source_0['ycentroid'],0)

    with open(f"{tmp_dir}calibrated_zipped_data/Calibrated_zipped_data_{f}.csv", 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['JD' ,'RA', 'DEC', 'Calibrated_Magnitude', 'Magnitude_error0'])
        writer.writerows(zip(jd_list, ra, dec, calibrated_mag, mag_err_0))
    
    print('file created for', f+1)   
end = time.time()
print('Execution took: ',end - start,' seconds')

In [ ]:
#Sorting calibrated magnitude files  wrt calibrated magnitude

f=0
csv_file = f"{tmp_dir}calibrated_zipped_data/Calibrated_zipped_data_{f}.csv"
# Read the CSV file
data = pd.read_csv(csv_file)

# Sort the data based on column 4
sorted_data = data.sort_values(by='Calibrated_Magnitude')

# Save the sorted data to a new CSV file
sorted_data.to_csv( f"{tmp_dir}Calibrated_zipped_data_{f}.csv", index=False) 

print("CSV file sorted successfully.")

### **Plotting Light Curves**
method 1

###### A. Ploting light curves for individual star
more is the magnitude, more is the associated error!

check if this trend is followed below, as we have sorted the calibrated magnitude file above.

In [ ]:
#Ploting light curve for single star

start =time.time()
warnings.filterwarnings("ignore")

files = sorted(glob(os.path.join(tmp_dir, "S-*cleaned.fits")))
# Read the reference file 
reference_file = pd.read_csv(f"{tmp_dir}Calibrated_zipped_data_0.csv")

sn = 11 #star number

reference_ra = reference_file['RA'][sn] 
reference_dec = reference_file['DEC'][sn]

matching_jd = []
matching_magnitude = []
matching_mag_error = []
for i in range(1, len(files)):  
    target_file = pd.read_csv(f"{tmp_dir}calibrated_zipped_data/Calibrated_zipped_data_{i}.csv")
    
    # Check if any RA and DEC values in the target file match with the reference values within the tolerance
    matches = ((target_file['RA'] - reference_ra).abs() <= 0.001) & ((target_file['DEC'] - reference_dec).abs() <= 0.001)
    
    # If there is a match, append the JD and magnitude values to the respective lists
    if matches.any():
        matching_jd.append(target_file.loc[matches, 'JD'].iloc[0])
        matching_magnitude.append(target_file.loc[matches, 'Calibrated_Magnitude'].iloc[0])
        matching_mag_error.append(target_file.loc[matches, 'Magnitude_error0'].iloc[0])
print('Number of data points:', len(matching_jd), len(matching_magnitude),len(matching_mag_error))
print(matches)

# Plot the magnitudes of the matching target files against their JD values
#plt.plot(matching_jd, matching_magnitude,'o')
plt.errorbar(matching_jd, matching_magnitude, yerr=matching_mag_error, color='blue',fmt='.', label='Data')
plt.xlabel('JD')
plt.ylabel('Magnitude')
plt.title('Light curve for %s star' %sn)
plt.show()

end = time.time()
print('Execution took: ',end - start,' seconds')

In [ ]:
#Ploting  light cuves for desired number of stars
# Read the reference file
reference_file = pd.read_csv(f"{tmp_dir}Calibrated_zipped_data_0.csv")
start = time.time()
start_index = 8 # Index of the first row to consider
end_index = 90 # Index of the last row to consider
# Iterate over the RA and DEC values in the reference file
for index, row in reference_file.iloc[start_index:end_index].iterrows():
    reference_ra = row['RA']
    reference_dec = row['DEC']

    # Initialize lists to store the matching target files' JD and magnitude values
    matching_jd = []
    matching_magnitude = []
    matching_mag_error = []
    
    for i in range(1, len(files)):  
        target_file = pd.read_csv(f"{tmp_dir}calibrated_zipped_data/Calibrated_zipped_data_{i}.csv")

        # Check if any RA and DEC values in the target file match with the reference values within the tolerance
        matches = (
            (target_file['RA'] - reference_ra).abs() <= 0.001
        ) & (
            (target_file['DEC'] - reference_dec).abs() <= 0.001
        )

        # If there is a match, append the JD and magnitude values to the respective lists
        if matches.any():
            matching_jd.append(target_file.loc[matches, 'JD'].iloc[0])
            matching_magnitude.append(target_file.loc[matches, 'Calibrated_Magnitude'].iloc[0])
            matching_mag_error.append(target_file.loc[matches, 'Magnitude_error0'].iloc[0])
    print('Number of data points:',len(matching_jd), len(matching_magnitude))
    print(index)
    
    # Plot the magnitudes of the matching target files against their JD values
    plt.errorbar(matching_jd, matching_magnitude, yerr=matching_mag_error, color='blue',markersize=1,fmt='.', label='Data')
    plt.xlabel('JD')
    plt.ylim(8.4,9.5)
    plt.ylabel('Magnitude')
    plt.title(f'Magnitudes of star with RA={reference_ra} and DEC={reference_dec}')
    plt.show()
end = time.time()
print('Execution took: ',end - start,' seconds')

###### B. Plotting Magnitude error vs Magnitude

In [ ]:
# Path to the CSV file
csv_file_path = tmp_dir + 'Calibrated_zipped_data_0.csv'

Magnitude_error_col = []
Calibrated_Magnitude_col = []

# Read the CSV file
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  
    
    for row in csv_reader:
        if row[3] != '':
            Magnitude_error_col.append(float(row[4]))  
            Calibrated_Magnitude_col.append(float(row[3]))  
            
# Plotting the data
plt.plot(Calibrated_Magnitude_col, Magnitude_error_col,'.', markersize=0.8)
plt.axhline(y=0.5, color='red', linestyle='--',label = 'y=0.5')
plt.xlabel('Calibrated Magnitude')
plt.ylim(-0.2,10)
plt.ylabel('Magnitude error')
plt.title('Magnitude error vs Magnitude')
plt.legend()
plt.show()

In [ ]:
#...with more details
# Load the CSV file into a pandas DataFrame
data = pd.read_csv('/home/aries/projetFolder/NGC6709/NGC6709_1/20211009/calibrated_zipped_data/Calibrated_zipped_data_0.csv')

calibrated_magnitude = data['Calibrated_Magnitude']
magnitude_error = data['Magnitude_error0']

mask = ~np.isnan(calibrated_magnitude)

calibrated_magnitude = calibrated_magnitude[mask]
magnitude_error = magnitude_error[mask]

# Define the number of bins for the histogram
num_bins = 1000
hist_bins = 100

hist, x_edges, y_edges = np.histogram2d(calibrated_magnitude, magnitude_error, bins=num_bins)

# Set up the figure and axes
fig, (ax1,ax2) = plt.subplots(2, 1, figsize=(6, 6), sharex=True)

fig.patch.set_facecolor('white')
ax2.set_facecolor('white')

ax1.hist(calibrated_magnitude, bins=hist_bins, color='lightblue', edgecolor='blue')
ax1.set_ylabel('Frequency')
ax1.tick_params(direction='in')
ax1.set_title('Histogram and contour', fontweight='bold')

# Create the contour plot with blue contours
contour = ax2.contourf(x_edges[:-1], y_edges[:-1], hist.T, levels=100, cmap='Blues')
ax2.set_ylim(-0, 8)
ax2.tick_params(direction='in')
plt.axvline(x=17, color='r', linestyle='--', label='x=17')
plt.legend()

ax2.set_xlabel('Calibrated Magnitude')
ax2.set_ylabel('Magnitude Error')

plt.subplots_adjust(hspace=0.06)
cbar = fig.add_axes([0.91, 0.15, 0.05, 0.3])
plt.colorbar(contour, cax=cbar, orientation='vertical').set_label(label='Density',size=10,weight='bold')

plt.show()

In [ ]:
#and more...
tmp_dir = '/home/aries/projetFolder/NGC6709/NGC6709_1/20211009/'
df = pd.read_csv(f'{tmp_dir}calibrated_zipped_data/Calibrated_zipped_data_0.csv')

# Remove rows with NaN values in the Calibrated_Magnitude column
df = df.dropna(subset=['Calibrated_Magnitude'])

# Plot density contour using seaborn
ax=sns.kdeplot(data=df, x='Calibrated_Magnitude', y='Magnitude_error0', fill=True, cmap='Blues',n_levels=25)
ax.text(11, 8, "Night 1",fontsize = 15,color = "Black",ha = "center", va = "center")

plt.xlabel('Calibrated Magnitude')
plt.ylabel('Magnitude Error')
plt.ylim(-2,10)
plt.title('Density Contour')
plt.show()

### **Plotting light curves from transposed data:** 
method 2 *for convenience and ease only*

In [ ]:
#Transpose of data
start = time.time()

directory_path = tmp_dir + "calibrated_zipped_data/"

# Read the reference file - file with max number of stars detected
reference_file = pd.read_csv("/home/aries/projetFolder/NGC6709/NGC6709_3/20211106/calibrated_zipped_data/Calibrated_zipped_data_234.csv")
target_Files = [(directory_path + f) for f in os.listdir(directory_path) if f.startswith('Calibrated_zipped_data')]
target_Files.sort(key=lambda x: int(x.split('_')[6].split('.')[0]))

# Filter reference file by 'Calibrated_magnitude' < 17
reference_file = reference_file[reference_file['Calibrated_Magnitude'] < 17]

# Iterate through each row in the reference file
start_index = 0  # Index of the first row to consider
end_index =  6500 # Index of the last row to consider

metadata = pd.DataFrame(columns=['File', 'RA', 'DEC'])

for index, row in reference_file.iloc[start_index:end_index].iterrows():
    reference_ra = row['RA']
    reference_dec = row['DEC']
    
    target_file_index = (index - start_index) % len(target_Files)
    target_file = target_Files[target_file_index]

    # Read the target file to obtain RA and DEC values
    target_data = pd.read_csv(target_file)
    target_ra = target_data['RA'].iloc[0]
    target_dec = target_data['DEC'].iloc[0]
    
    # Create an empty DataFrame to store the results for each matching file
    matching_results = pd.DataFrame(columns=['JD', 'Calibrated_Magnitude', 'Magnitude_error0'])

    # Add the reference row to the matching_results DataFrame
    matching_results.loc[len(matching_results)] = row[['JD', 'Calibrated_Magnitude', 'Magnitude_error0']]
    unique_jd_values = set()

    unique_jd_values.add(row['JD'])

    # Iterate through each file (excluding the reference file)
    for i in range(0,len(target_Files)):
        filename = f"{tmp_dir}calibrated_zipped_data/Calibrated_zipped_data_{i}.csv"  # Assuming the file names are in the format '1.csv', '2.csv', ...
        file_data = pd.read_csv(filename)

        matching_data = file_data[np.sqrt((np.square(file_data['RA'] - reference_ra)) + (np.square(file_data['DEC'] - reference_dec))) < 1.41/3600]

        # Iterate through the matching data rows
        for _, matching_row in matching_data.iterrows():
            jd_value = matching_row['JD']

            if jd_value not in unique_jd_values:
                # Add a new row to the matching_results DataFrame
                matching_results.loc[len(matching_results)] = matching_row[['JD', 'Calibrated_Magnitude', 'Magnitude_error0']]
                # Add the JD value to the set of unique JD values
                unique_jd_values.add(jd_value)

    # Save the matching results to a CSV file
    matching_results.to_csv(f"{tmp_dir}transposed_data/Mag_file_star_{index}.csv", index=False)
    print('File created for:', index)
    metadata.loc[len(metadata)] = [f"{tmp_dir}transposed_data/Mag_file_star_{index}.csv", reference_ra, reference_dec]

    
metadata_filename = f"{tmp_dir}metadata.csv"
metadata.to_csv(metadata_filename, index=False)

end = time.time()
print('Execution took:', end - start, 'seconds')

In [ ]:
#PLOTING LIGHT CURVE AND HISTOGRAM OF DEVIATION FROM MEAN using transposed data

# Read the reference file
reference_file = pd.read_csv(f"{tmp_dir}Calibrated_zipped_data_0.csv")
directory_path = tmp_dir + "transposed_data/"
csv_files = [(directory_path + f) for f in os.listdir(directory_path) if f.startswith('Mag_file_star')]
csv_files.sort(key=lambda x: int(x.split('_')[5].split('.')[0]))

df = pd.read_csv(csv_files[80]) #any file from files

magni_tude = [df.iloc[:,1]]
print(np.mean(magni_tude))
print(magni_tude)
diff1=[]
for i in range(100):
    diff =float(df.iloc[i, 1] - np.mean(magni_tude))
    diff1.append(diff)
        
plt.plot(df.iloc[:, 0], df.iloc[:, 1], 'o')
  # Adjust the width and height as needed
plt.xlabel('JD')
plt.ylabel('Magnitude')
plt.title('Light curve')
plt.axhline(y=np.mean(magni_tude), color='r', linestyle='--', label='mean')
plt.legend()
plt.show()

print(diff1)
if diff1 != 'nan':
    plt.hist(diff1, bins=50)
    plt.xlabel('values')
    plt.ylabel('frequency')
    plt.title('Deviation from mean')
#plt.axvline(x=np.mean(magni_tude), color='r', linestyle='--',label='mean')

    plt.show()

### **Differential light curves**

###### A. Minimum difference using transposed data

In [ ]:

def check_first_column(csv_files):
    first_column_values = set()
    first_column_lengths = set()
    
    for file in csv_files:
        with open(file, 'r') as csv_file:
            reader = csv.reader(csv_file)
            first_column = []
            
            for row in reader:
                first_column.append(row[0])
            
            first_column_values.add(tuple(first_column))
            first_column_lengths.add(len(first_column))
    
    return len(first_column_values) == 1 and len(first_column_lengths) == 1

# Directory path containing the CSV files
directory_path = tmp_dir + "transposed_data/"

csv_files = []
for filename in os.listdir(directory_path):
    if filename.startswith('Mag_file_star') and filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        csv_files.append(file_path)

# Check if all CSV files have the same first column
if check_first_column(csv_files):
    print("All CSV files have the same first column.")
else:
    print("CSV files have different values or lengths in the first column.")


In [ ]:
start = time.time()

tmp_dir = '/home/aries/projetFolder/NGC6709/NGC6709_1/20211009/'

# Set the directory path where the CSV files are located
directory_path = f'{tmp_dir}transposed_data/'
# Set the chosen number for the range
x = 14

# Get a list of all CSV files in the directory
csv_files = [(directory_path + f) for f in os.listdir(directory_path) if f.startswith('Mag_file_')]
print(len(csv_files))
# Create an empty list to store the filtered file names
filtered_files = []

# Iterate through each CSV file
for file in csv_files:
    # Read the CSV file
    data = pd.read_csv(file)
    
    # Calculate the lower and upper bounds for the range
    lower_bound = x - 1
    upper_bound = x + 1
    
    # Filter the data based on the condition
    filtered_data = data[(data['Calibrated_Magnitude'] >= lower_bound) & (data['Calibrated_Magnitude'] <= upper_bound)]
    
    # Calculate the percentage of rows within the range
    percentage_within_range = len(filtered_data) / len(data) * 100

    # Check if the percentage meets the condition of 90%
    if percentage_within_range >= 90:
        filtered_files.append(file)
print(len(filtered_files))

# Print the filtered file names
filtered_files.sort(key=lambda x: int(x.split('_')[5].split('.')[0]))
for file in filtered_files:
    print(file)
    
end = time.time()
print('Execution took:', end - start, 'seconds')

In [ ]:
start=time.time()
folder_path = tmp_dir + 'transposed_data'  # Replace with the actual folder path

# # Get a list of all CSV files in the folder
# csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
csv_files= filtered_files
csv_files.sort(key=lambda x: int(x.split('_')[5].split('.')[0]))
#csv_files=csv_files[1:5]
print(len(csv_files))
# Generate all possible combinations of two CSV files
# Generate all possible combinations of two CSV files
file_combinations = list(combinations(csv_files, 2))

# Initialize variables to track minimum standard deviation and corresponding file pair
min_std_dev = float('inf')
min_std_dev_files = None
sigma_list=[]
i=0
# Read and process each file combination
for file_pair in file_combinations:
    reference_file = file_pair[0]
    compare_file = file_pair[1]
    
    reference_file_path = os.path.join(folder_path, reference_file)
    compare_file_path = os.path.join(folder_path, compare_file)
    
    reference_df = pd.read_csv(reference_file_path)
    compare_df = pd.read_csv(compare_file_path)
    
    # Merge the reference and compare DataFrames based on the 'JD' column
    merged_df = pd.merge(reference_df, compare_df, on='JD', how='inner')
    
    # Perform subtraction between corresponding 'Calibrated_Magnitude' values and store the result in a list
    subtraction_list = merged_df['Calibrated_Magnitude_x'] - merged_df['Calibrated_Magnitude_y']
    
    # Calculate the standard deviation of the subtraction list
    std_dev = subtraction_list.std()
    
    # Append the standard deviation to the 'sigma_list'
    sigma_list.append(std_dev)
    
    # Check if the current standard deviation is the minimum
    if std_dev < min_std_dev:
        min_std_dev = std_dev
        min_std_dev_files = (reference_file, compare_file)
    
    # Print the subtraction list and its standard deviation for the current file combination
    print(f"Subtraction list for {reference_file} and {compare_file}:")
    #print(subtraction_list)
    print(f"Standard Deviation: {std_dev}")
    print()
    i +=1
# Print the file pair corresponding to the minimum standard deviation
print("File pair with minimum standard deviation:")
print(min_std_dev_files)
print("Minimum Standard Deviation:")
print(min_std_dev)
print('total combinations', i)

end = time.time()
print('Execution took:', end - start, 'seconds')

In [ ]:
# Sort the list in ascending order
sorted_numbers = sorted(sigma_list)

# Extract the top 10 least elements
top_10_least = sorted_numbers[:30]

# Print the top 10 least elements
print("Top 10 least numbers:")
for number in top_10_least:
    print(number)

###### B. **PLOTING DIFFERENTIAL LIGHT CURVES** 

In [ ]:
#PLOTING LIGHT CURVES OF STARS WITH MAG RANGE (+-1) of x

# check for bin size
start = time.time()

# Read the first CSV file
df1 = pd.read_csv(f'{tmp_dir}transposed_data/Mag_file_star_245.csv')
# plt.plot(df1.iloc[:, 0],df1.iloc[:, 1],'.')
# plt.show()

df2 = pd.read_csv(f'{tmp_dir}transposed_data/Mag_file_star_270.csv')
for i in filtered_files[:5]: #here enter only chosen bin 
    
    df3 = pd.read_csv(i)
    # Extract the first and second columns from both dataframes
    x = df2.iloc[:, 0]  # 1st column of file1 (assuming index starts from 0) 
    if len(df1)==len(df2)==len(df3):
        diff1 = df3.iloc[:, 1] - df1.iloc[:, 1]  
        diff2 = df2.iloc[:, 1] - df3.iloc[:, 1]
        diff3 = df1.iloc[:, 1] - df2.iloc[:, 1]
               
        #plt.axhline(y=np.mean(diff3), color='r', linestyle='--', label='mean of C1-C2')
        plt.plot(x, diff1, '.', label='T-C1')
        plt.plot(x, diff2,'.',label='T-C2')    
        plt.plot(x, diff3 ,'.',label='C1-C2')
        
        # Add labels and title
        plt.ylabel('Differential magnitude')
        plt.xlabel('JD')
        plt.title(f'Difference of Magnitude {i}')
        plt.legend()
        #plt.xlim(2459497.1037037, 2459497.17280815)
        plt.show()
    
end = time.time()
print('Execution took: ',end - start,' seconds')

In [ ]:
# Read the CSV file

for file_name in matched_file_paths:
    data = pd.read_csv(file_name)
    data = data.sort_values('JD')
    print(file_name)
    print(type(data['JD']))
    frame_list = data['JD'].tolist()
    print(type(frame_list))
    jd_lists = make_jd_list(file_name)
    
    bin_size = 30
    num_bins = len(data) // bin_size
    print(bin_size,num_bins)
    binned_data = pd.DataFrame(columns=['JD', 'Calibrated_Magnitude', 'Magnitude_error0'])

    for i in range(num_bins):
    # Determine the start and end indices for each bin
        start_index = i * bin_size
        end_index = start_index + bin_size

    # Extract the data points within the current bin
        bin_points = data.iloc[start_index:end_index]
        std_dev = np.std(np.array(jd_lists[i], dtype=float))
    # Calculate the mean values for the bin
        mean_jd = bin_points['JD'].mean()
        mean_mag = bin_points['Calibrated_Magnitude'].mean()
        mean_error = bin_points['Magnitude_error0'].mean()
    
    # Append the binned data to the DataFrame
        bin_data = pd.DataFrame({'JD': [mean_jd],'JD_error': [std_dev], 'Calibrated_Magnitude': [mean_mag], 'Magnitude_error0': [mean_error]})
        binned_data = pd.concat([binned_data, bin_data], ignore_index=True)

    diff_binned_data =[]
    for i in range(0,len(binned_data['JD'])):
        a = binned_data['JD'][i] - binned_data['JD'][0]
        diff_binned_data.append(a)
    
#     print(binned_data['JD_error'])
# Plot the binned data with error bars
    plt.errorbar(binned_data['JD'] , binned_data['Calibrated_Magnitude'],xerr=binned_data['JD_error'], yerr=binned_data['Magnitude_error0'],markersize=1, fmt='o')

    plt.xlabel('JD')
    plt.ylabel('Calibrated Magnitude')
    plt.title('Plot of Calibrated Magnitude vs. JD')

# Add a legend
#plt.legend()

# Show the plot
    plt.show()


In [ ]:
# Read the CSV file
for r in matched_file_paths:
    csv_file = r
    print(csv_file)
    data = pd.read_csv(csv_file)
    data = data.sort_values('JD')

    jd_lists = make_jd_list(csv_file)
    print(len(jd_lists[0]))

    binned_data = pd.DataFrame(columns=['JD','JD_error', 'Calibrated_Magnitude', 'Magnitude_error0'])

# Calculate the standard deviation
    std_dev = np.std(np.array(jd_lists[0], dtype=float))
    n = len(jd_lists[0])
    std_error = std_dev / np.sqrt(n)
    print(std_dev)
    print(r)
# Iterate over the bins
    for i in range(num_bins):
    # Determine the start and end indices for each bin
        start_index = i * bin_size
        end_index = start_index + bin_size

    # Extract the data points within the current bin
        bbin_points = data.iloc[start_index:end_index]
        std_dev = np.std(np.array(jd_lists[i], dtype=float))
    # Calculate the mean values for the bin
        mean_jd = bin_points['JD'].mean()
        mean_mag = bin_points['Calibrated_Magnitude'].mean()
        mean_error = bin_points['Magnitude_error0'].mean()
    
        # Append the binned data to the DataFrame
        bin_data = pd.DataFrame({'JD': [mean_jd],'JD_error': [std_dev], 'Calibrated_Magnitude': [mean_mag], 'Magnitude_error0': [mean_error]})
        binned_data = pd.concat([binned_data, bin_data], ignore_index=True)
    
    print(binned_data['JD'])
    diff_binned_data =[]
    for i in range(len(binned_data['JD'])):
        a = binned_data['JD'][i] - binned_data['JD'][0]
        diff_binned_data.append(a)
        
    
# Plot the binned data with error bars
    plt.errorbar(diff_binned_data, binned_data['Calibrated_Magnitude'],xerr=binned_data['JD_error'], yerr=binned_data['Magnitude_error0'],markersize=5, fmt='o')
#     for r in range(len(binned_data['Calibrated_Magnitude'])):
#         if binned_data['Calibrated_Magnitude'][r] > 17:
#             print(binned_data['Calibrated_Magnitude'][r])
        
    print(np.mean(binned_data['Calibrated_Magnitude']))
    plt.axhline(y=np.mean(binned_data['Calibrated_Magnitude']),color='r')
#plt.errorbar(binned_data['JD'], binned_data['Calibrated_Magnitude'],xerr=std_dev, yerr=binned_data['Magnitude_error0'],markersize=0.5, fmt='o')
    plt.xlabel('JD in days')
    plt.ylabel('Calibrated Magnitude')
    plt.title('Binned Data of a star from 1st night')
    plt.show()

In [ ]:
#ERROR BAR INCLUDED

start = time.time()
# Read the first CSV file
df1 = pd.read_csv(f'{tmp_dir}transposed_data/Mag_file_star_396.csv')
plt.plot(df1.iloc[:, 0], df1.iloc[:, 1], '.')

# Read the second CSV file
df2 = pd.read_csv(f'{tmp_dir}transposed_data/Mag_file_star_922.csv')

for i in filtered_files:  # here enter only chosen bin
    df3 = pd.read_csv(i)

    # Extract the columns from all dataframes
    x = df2.iloc[:, 0]  # 1st column of file2 (assuming index starts from 0)
    
    if len(df1) == len(df2) == len(df3):
        diff1 = df3.iloc[:, 1] - df1.iloc[:, 1]
        diff2 = df3.iloc[:, 1] - df2.iloc[:, 1]
        diff3 = df1.iloc[:, 1] - df2.iloc[:, 1]
        
        error1 = df3.iloc[:, 2]  # Magnitude_error0 column of file3
        error2 = df2.iloc[:, 2]  # Magnitude_error0 column of file2
        error3 = df1.iloc[:, 2]  # Magnitude_error0 column of file1
        
        error_diff1 = np.sqrt(error1**2 + error2**2)
        error_diff2 = np.sqrt(error2**2 + error3**2)
        error_diff3 = np.sqrt(error3**2 + error1**2)
        
        plt.errorbar(x, diff1, yerr=error_diff1,markersize=2 ,fmt='.', label='T-C1')
        plt.errorbar(x, diff2, yerr=error_diff2, markersize=2 ,fmt='.', label='T-C2')
        plt.errorbar(x, diff3, yerr=error_diff3,markersize=1 , fmt='.', label='C1-C2')

        plt.ylabel('Differential magnitude')
        plt.xlabel('JD')
        plt.title(f'Difference of Magnitude {i}')
        plt.legend()

        # Display the plot
        plt.show()
        
end = time.time()
print('Execution took: ',end - start,' seconds')

In [ ]:
#Binnning and errorbar

start = time.time()

# Read the first CSV file
df1 = pd.read_csv(f'{tmp_dir}transposed_data/Mag_file_star_295.csv')

# Read the second CSV file
df2 = pd.read_csv(f'{tmp_dir}transposed_data/Mag_file_star_297.csv')

# Define binning parameters
bin_size = 25  # Number of data points per bin

for i in filtered_files:  # here enter only chosen bin
    df3 = pd.read_csv(i)

    
    x = df2.iloc[:, 0]  

    if len(df1) == len(df2) == len(df3):
        diff1 = df3.iloc[:, 1] - df1.iloc[:, 1]
        diff2 = df3.iloc[:, 1] - df2.iloc[:, 1]
        diff3 = df1.iloc[:, 1] - df2.iloc[:, 1]

        
        error1 = df3.iloc[:, 2]  
        error2 = df2.iloc[:, 2]  
        error3 = df1.iloc[:, 2]  

        # Calculate the error in the difference using error propagation
        error_diff1 = np.sqrt(error1**2 + error2**2)
        error_diff2 = np.sqrt(error2**2 + error3**2)
        error_diff3 = np.sqrt(error3**2 + error1**2)
    
        num_bins = len(x) // bin_size
        x_binned = np.array_split(x, num_bins)

        # Initialize arrays to store binned values and errors
        diff1_binned = np.zeros(num_bins)
        diff2_binned = np.zeros(num_bins)
        diff3_binned = np.zeros(num_bins)
        error_diff1_binned = np.zeros(num_bins)
        error_diff2_binned = np.zeros(num_bins)
        error_diff3_binned = np.zeros(num_bins)

        # Bin the values and errors
        for j, bin_values in enumerate(x_binned):
            diff1_binned[j] = np.mean(diff1[bin_values.index])
            diff2_binned[j] = np.mean(diff2[bin_values.index])
            diff3_binned[j] = np.mean(diff3[bin_values.index])
            error_diff1_binned[j] = np.sqrt(np.mean(error_diff1[bin_values.index]**2))
            error_diff2_binned[j] = np.sqrt(np.mean(error_diff2[bin_values.index]**2))
            error_diff3_binned[j] = np.sqrt(np.mean(error_diff3[bin_values.index]**2))

        # Plot the scatter plot with error bars
        plt.errorbar(np.arange(num_bins) * bin_size, diff1_binned, yerr=error_diff1_binned, markersize=5, fmt='.', label='T-C1')
        plt.errorbar(np.arange(num_bins) * bin_size, diff2_binned, yerr=error_diff2_binned, markersize=5, fmt='.', label='T-C2')
        plt.errorbar(np.arange(num_bins) * bin_size, diff3_binned, yerr=error_diff3_binned, markersize=3, fmt='.', label='C1-C2')
        plt.axhline(y=np.mean(diff3_binned), color='r')
        plt.ylabel('Differential magnitude')
        plt.xlabel('JD')
        plt.title(f'Difference of Magnitude {i}')
        plt.legend()
        plt.show()

end = time.time()
print('Execution took:', end - start, 'seconds')


### **Phase-Magnitude diagrams**

In [ ]:
# List of metadata file names
metadata_files = ['/home/aries/projetFolder/NGC6709/NGC6709_1/20211009/metadata.csv',
             '/home/aries/projetFolder/NGC6709/NGC6709_2/20211105/metadata.csv', 
             '/home/aries/projetFolder/NGC6709/NGC6709_3/20211106/metadata.csv', 
             '/home/aries/projetFolder/NGC6709/NGC6709_4/20211116/metadata.csv', 
             '/home/aries/projetFolder/NGC6709/NGC6709_5/20211117/metadata.csv',
            '/home/aries/projetFolder/NGC6709/NGC6709_6/20211120/metadata.csv',
            '/home/aries/projetFolder/NGC6709/NGC6709_7/20220319/metadata.csv',
            '/home/aries/projetFolder/NGC6709/NGC6709_8/20220328/metadata.csv',
            '/home/aries/projetFolder/NGC6709/NGC6709_9/20220410/metadata.csv',
            '/home/aries/projetFolder/NGC6709/NGC6709_12/20221031/metadata.csv']

for k in range(1,20):
    #k= row of metadata.csv
    # Initialize an empty list to store the file names
    RA =[]
    DEC =[]
    file_names = []
    # Iterate over each metadata file
    for metadata_file in metadata_files:
        # Read the metadata.csv file
        metadata_df = pd.read_csv(metadata_file)

        # Get the file paths from the 3rd row under the 'file' column
        file_paths = metadata_df.loc[k, 'File']
        Ra, Dec = metadata_df.loc[k, 'RA'], metadata_df.loc[k, 'DEC']
        # Split the file paths into a list
        file_paths_list = file_paths.split(',')
        #Ra_list, Dec_list = Ra.split(','), Dec.split(',')
        # Append the third file path to the file_names list
        file_names.append(file_paths_list)  # Assuming the third file path is required
        RA.append(Ra) 
        DEC .append(Dec)
    file_names = list(itertools.chain(*file_names))
    print(RA, DEC)
    # Function to perform sigma clipping for a given DataFrame
    def perform_sigma_clipping(df):
        mean = df['Calibrated_Magnitude'].mean()
        std = df['Calibrated_Magnitude'].std()
        cutoff = 3 * std
        df = df[(df['Calibrated_Magnitude'] >= mean - cutoff) & (df['Calibrated_Magnitude'] <= mean + cutoff)]
        return df
    
    combined_dfs = []

    for file_name in file_names:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_name)
        df = perform_sigma_clipping(df)

        combined_dfs.append(df)

    # Concatenate all the DataFrames in the combined list
    combined_df = pd.concat(combined_dfs)
    print(type(combined_df['JD']))

    x, y =(RA[0]), (DEC[0])
    
    # Plotting the data
    plt.errorbar(combined_df['JD'], combined_df['Calibrated_Magnitude'],yerr=combined_df['Magnitude_error0'], markersize=5, fmt='.')
    plt.xlabel('JD in days')
    plt.ylabel('Calibrated Magnitude')
    plt.title('Light curve for RA={} Dec={}'.format(x, y)) 
    #plt.xlim(2459497.1036500,2459497.17280000)
    plt.grid(True)
    plt.show()

In [ ]:
# Load and preprocess the data
jd = combined_df['JD']  
magnitudes = combined_df['Calibrated_Magnitude'] 
mag_err = combined_df['Magnitude_error0']

# Calculate the Lomb-Scargle periodogram
frequency, power = LombScargle(jd, magnitudes).autopower()

dominant_period = 1 / frequency[np.argmax(power)]
phase = ((jd - jd[0]) / dominant_period) % 1

amplitude = np.max(magnitudes) - np.min(magnitudes)
dominant_period = round(dominant_period,5)
amplitude = round(amplitude,5)

# Plot the folded light curve
plt.text(0.5,8,f"P = {dominant_period} days", fontsize =12) 
plt.text(0.5,8.1,f"A = {amplitude}", fontsize =12)

plt.errorbar(phase, magnitudes, yerr=mag_err, markersize=1,fmt='.', color='black')
plt.xlabel('Phase')
plt.ylabel('Magnitude')
plt.title(f'RA = {RA[0]}, DEC = {DEC[0]}')
plt.gca().invert_yaxis()
plt.grid()
plt.show()

print("Period is", dominant_period, 'days')
print("Amplitude:", amplitude)